<a href="https://colab.research.google.com/github/yemauricio/caramelo/blob/main/etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejecución Manual ETL caramelo

### Clonar repositorio

In [13]:
! git clone https://github.com/yemauricio/caramelo.git

Cloning into 'caramelo'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 110 (delta 37), reused 68 (delta 10), pack-reused 0
Receiving objects: 100% (110/110), 1.10 MiB | 18.10 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [14]:
cd caramelo/

/content/caramelo


In [15]:
!pwd

/content/caramelo


In [19]:
%run -i 'etl.py'
%timeit

ERROR:root:File `'etl.py'` not found.


In [18]:
cd resultado/

/content/caramelo/resultado


In [17]:
ls -la

total 80
drwxr-xr-x 7 root root  4096 Jan 25 23:25 ./
drwxr-xr-x 1 root root  4096 Jan 25 23:25 ../
drwxr-xr-x 2 root root  4096 Jan 25 23:25 catalogos/
-rw-r--r-- 1 root root  4317 Jan 25 23:25 etl.ipynb
-rw-r--r-- 1 root root 16439 Jan 25 23:25 etl.py
-rw-r--r-- 1 root root   132 Jan 25 23:25 etl_resultado.bat
drwxr-xr-x 8 root root  4096 Jan 25 23:25 .git/
drwxr-xr-x 2 root root  4096 Jan 25 23:25 insumo/
-rw-r--r-- 1 root root 11357 Jan 25 23:25 LICENSE
-rw-r--r-- 1 root root   462 Jan 25 23:25 README.md
-rw-r--r-- 1 root root    29 Jan 25 23:25 requirements.txt
drwxr-xr-x 2 root root  4096 Jan 25 23:27 resultado/
drwxr-xr-x 2 root root  4096 Jan 25 23:25 vistas/
